In [1]:
import numpy as np
from collections import Counter
from copy import deepcopy

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

In [11]:
def four_in_a_row(bitboard):
    if (bitboard & (bitboard >> 6) & (bitboard >> 12) & (bitboard >> 18) != 0):
         return True # diagonal \
    if (bitboard & (bitboard >> 8) & (bitboard >> 16) & (bitboard >> 24) != 0):
         return True # diagonal /
    if (bitboard & (bitboard >> 7) & (bitboard >> 14) & (bitboard >> 21) != 0):
         return True # horizontal
    if (bitboard & (bitboard >> 1) & (bitboard >>  2) & (bitboard >>  3) != 0):
         return True # vertical
    return False

def _mc(bitmap):
    while bitmap.valid_moves():
        c = np.random.choice(bitmap.valid_moves())
        bitmap.play(c)
        if four_in_a_row(bitmap.bitboard[0]):
            return 1
        if four_in_a_row(bitmap.bitboard[1]):
            return -1
    return 0


class Bitmap():
    def __init__(self):
        self.counter = 0 
        self.bitboard = [0, 0]
        self.height = [0,7,14,21,28,35,42]
        self.moves = []
    
    def play(self, idx):
        move = 1 << self.height[idx]
        self.height[idx] += 1
        self.bitboard[self.counter & 1] ^= move
        self.moves.append(idx)
        self.counter += 1
    
    def take_back(self):
        self.counter -= 1
        idx = self.moves.pop()
        self.height[idx] -= 1
        move = 1 << self.height[idx]
        self.bitboard[self.counter & 1] ^= move
    
    def check_end(self):
        if(four_in_a_row(self.bitboard[0])):
            return 1
        elif(four_in_a_row(self.bitboard[1])):
            return -1
        elif (self.bitboard[0] ^ self.bitboard[1]) == 279258638311359:
            return 0

    def valid_moves(self):
        moves = []
        top = int('1000000_1000000_1000000_1000000_1000000_1000000_1000000', 2)
        for idx in range(0, NUM_COLUMNS):
            if( top & (1 << self.height[idx]) == 0):
                moves.append(idx)
        return moves

    def montecarlo(self):
        montecarlo_samples = 100
        cnt = Counter(_mc(deepcopy(self)) for _ in range(montecarlo_samples))
        return (cnt[1] - cnt[-1]) / montecarlo_samples


    def eval_board(self):
        if four_in_a_row(self.bitboard[0]):
            # Alice won
            return 1
        elif four_in_a_row(self.bitboard[1]):
            # Bob won
            return -1
        else:
            # Not terminal, let's simulate...
            return self.montecarlo()

    def print_board(self):
        npboard = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
        p1 = "{0:b}".format(self.bitboard[0])
        p2 = "{0:b}".format(self.bitboard[1])
        idx = 0
        for k in range(len(p1)-1, -1, -1):
            i = int(idx/7)
            j = idx%7
            if int(p1[k])==1:
                npboard[i, j] = 1
            idx += 1
        idx = 0
        for k in range(len(p2)-1, -1, -1):
            i = int(idx/7)
            j = idx%7
            if int(p2[k])==1:
                npboard[i, j] = -1
            idx +=1
        print(npboard)

bm = Bitmap()
bm.play(3)
bm.play(3)
bm.play(3)
bm.play(3)
print(bm.bitboard[0], bm.bitboard[1])


10485760 4194304


In [14]:
class minmax_alphabeta_bitmap:
    def __init__(self):
        self.state = Bitmap()
        self.player_turn = 1

    def max(self, alpha, beta, depth):
        result = self.state.check_end()
        if result != None or depth<=0:
            if result == 1:
                return (1, 0)
            elif result == -1:
                return (-1, 0)
            elif result == 0:
                return (0, 0)
            else:
                return (self.state.eval_board(), 0)
        max_value = -2
        max_idx = None
        for i in self.state.valid_moves():
            self.state.play(i)
            self.player_turn = -self.player_turn
            (value, min_idx) = self.min(alpha, beta, depth-1)
            if value > max_value:
                max_value = value
                max_idx = i
            self.state.take_back()
            self.player_turn = -self.player_turn
            if max_value >= beta:
                return (max_value, max_idx)
            if max_value > alpha:
                alpha = max_value
        return (max_value, max_idx)
            
    
    def min(self, alpha, beta, depth):
        result = self.state.check_end()
        if result != None or depth<=0:
            if result == 1:
                return (1, 0)
            elif result == -1:
                return (-1, 0)
            elif result == 0:
                return (0, 0)
            else:
                return (self.state.eval_board(), 0)
        min_value = 2
        min_idx = None
        for i in self.state.valid_moves():
            self.state.play(i)
            self.player_turn = -self.player_turn
            (value, max_idx) = self.max(alpha, beta, depth-1)
            if value < min_value:
                min_value = value
                min_idx = i
            self.state.take_back()
            self.player_turn = -self.player_turn
            if min_value <= alpha:
                return (min_value, min_idx)
            if min_value < beta:
                beta = min_value
        return (min_value, min_idx)

    def look_up_table(self):
        if self.state.counter == 0:
            return 3
        elif (self.state.bitboard[0] == 2097152 and self.state.bitboard[1] == 0 ) or (self.state.bitboard[1] == 2097152 and self.state.bitboard[0] == 0):
            return 3
        elif (self.state.bitboard[0] == 2097152 and self.state.bitboard[1] == 4194304 ) or (self.state.bitboard[1] == 2097152 and self.state.bitboard[0] == 4194304):
            return 3
        elif (self.state.bitboard[0] == 10485760 and self.state.bitboard[1] == 4194304 ) or (self.state.bitboard[1] == 10485760 and self.state.bitboard[0] == 4194304):
            return 3
        return None
    
    def play(self, depth):
        turn =1
        while True:
            self.result = self.state.check_end()
            if self.result != None:
                if self.result == 1:
                    print('The winner is Player 1')
                elif self.result == -1:
                    print('The winner is Player -1')
                elif self.result == 0:
                    print("It's a tie!")
                return
            
            index = self.look_up_table()
            if self.player_turn == -1:
                if index == None:
                    (m, index) = self.min(-2, 2, depth)
            else:
                if index == None:
                    (m, index) = self.max(-2, 2, depth)
            print('Turn', turn , 'Player:',self.player_turn)
            turn += 1
            self.state.play(index)
            self.state.print_board()
            self.player_turn = -self.player_turn

In [15]:
game = minmax_alphabeta_bitmap()
game.play(3)

Turn 1 Player: 1
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Turn 2 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Turn 3 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Turn 4 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Turn 5 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Turn 6 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  1 -1  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Turn 7 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0 